In [ ]:
import pandas as pd 

clean_df = pd.read_csv('clean_tabla.csv', index_col=0)

In [ ]:
import umap
import umap.plot
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline

df = clean_df.copy()
df.head(5)

In [ ]:
to_keep = ['nivel_desemp_matematica','nivel_desemp_lengua', 'nivel_desemp_ciencias_sociales', 
           'nivel_desemp_ciencias_naturales', 'mate_nc_na', 'lengua_nc_na', 'sociales_nc_na', 'naturales_nc_na'
]
data = df[to_keep].copy()

In [ ]:
data = pd.merge(data, pd.get_dummies(data.nivel_desemp_lengua, 'lengua'), left_index=True, right_index=True)
data = pd.merge(data, pd.get_dummies(data.nivel_desemp_matematica, 'mate'), left_index=True, right_index=True)
data = pd.merge(data, pd.get_dummies(data.nivel_desemp_ciencias_sociales, 'ciencias_sociales'), left_index=True, right_index=True)
data = pd.merge(data, pd.get_dummies(data.nivel_desemp_ciencias_naturales, 'ciencias_naturales'), left_index=True, right_index=True)
data

In [ ]:
not_nan = (df.mate_nc_na == 0) & (df.lengua_nc_na == 0) & (df.naturales_nc_na == 0) & (df.sociales_nc_na == 0) #ONLY MATE
train = data[not_nan].copy()

test = data[~not_nan].copy()

In [ ]:
def label(row, suffix):
  if row[f"{suffix}_por_debajo_del_basico"] > 0:
    return 0
  if row[f"{suffix}_basico"] > 0:
    return 1
  if row[f"{suffix}_satisfactorio"] > 0:
    return 2
  if row[f"{suffix}_avanzado"] > 0: 
    return 3
  else:
    return -1


In [ ]:
train['label'] = data.apply(lambda x: label(x, 'mate'), axis=1)
train

In [ ]:
to_drop = ['nivel_desemp_matematica','nivel_desemp_lengua', 'nivel_desemp_ciencias_sociales', 
           'nivel_desemp_ciencias_naturales', 'mate_nc_na', 'lengua_nc_na', 'sociales_nc_na', 'naturales_nc_na',
           'mate_por_debajo_del_basico','mate_satisfactorio','mate_avanzado', 'mate_basico'
]

train.drop(columns=to_drop, inplace=True)

In [ ]:
train

In [ ]:
train = train.sample(20000).copy()
train

In [ ]:
%%time

def train_UMAP(df, n_neighbors=15, min_dist=0.1,spread=1.0, repulsion_str = 1, metric='euclidean', n_epochs=None):
  mapper = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist,metric=metric,n_epochs=n_epochs, spread=spread, repulsion_strength=repulsion_str)
  return mapper.fit(df[df.columns.difference(['label'])], y = df.label)

UMAP_model = train_UMAP(df=train,n_neighbors=300,min_dist=0.5, spread= 1,repulsion_str=1,n_epochs=300, metric='dice')

def plot_umap(model=UMAP_model, labels=train.label):
    umap.plot.points(model,labels=labels, background='black')


plot_umap()

In [ ]:
%%time
fig = plt.figure()
size = [0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
for i in range(len(size)):
  UMAP_model = train_UMAP(df=train,n_neighbors=500,min_dist=0.2, spread=size[i],repulsion_str=2,n_epochs=300, metric='canberra')
  plot_umap(UMAP_model, train.label)

In [ ]:
%%time
fig = plt.figure()
size = [1.5, 2, 2.5, 3]
for i in range(len(size)):
  UMAP_model = train_UMAP(df=train,n_neighbors=500,min_dist=0.2, spread=0.4,repulsion_str=size[i],n_epochs=300, metric='canberra')
  plot_umap(UMAP_model, train.label)

In [ ]:
%%time
fig = plt.figure()
size = [1, 1.5, 2, 2.5, 3]
for i in range(len(size)):
  UMAP_model = train_UMAP(df=train,n_neighbors=500,min_dist=0.5, spread=size[i],repulsion_str=3,n_epochs=500, metric='canberra')
  plot_umap(UMAP_model, train.label)

In [ ]:
%%time
size = [15, 30, 45, 60, 75, 90]
for i in range(len(size)):
  UMAP_model = train_UMAP(df=train,n_neighbors=size[i],min_dist=1, spread=3,repulsion_str=1,n_epochs=300, metric='canberra')
  plot_umap(UMAP_model, train.label)

In [ ]:
%%time
size = [100, 200, 300, 400, 500]
for i in range(len(size)):
  UMAP_model = train_UMAP(df=train,n_neighbors=size[i],min_dist=1, spread=3,repulsion_str=1,n_epochs=350, metric='canberra')
  plot_umap(UMAP_model, train.label)

In [ ]:
new_columns = []

for name in column_names:
    W = NMF_model.transform(test.iloc[:,test.columns.str.match(f"{name}_\w+")].copy())
    new_columns.append(W.flatten().tolist())

schema=dict(zip(column_names, new_columns))

test = pd.DataFrame(data=schema, index=test.index)
test

In [ ]:
scaled_values = quant_scaler.transform(test.values)
test = pd.DataFrame(data=scaled_values, index=test.index, columns=test.columns)
test.describe()

In [ ]:
test = test.copy().sample(1000)
test

In [ ]:
result = UMAP_model.transform(test.values)